In [5]:
import tweepy
from kafka import KafkaProducer
from json import dumps
import time
import json

ERROR! Session/line number was not unique in database. History logging moved to new session 943


In [6]:
#Authenticate on Twitter

consumer_key = "dmn51aowr3bYzIWz3ELgARN5x"
consumer_secret = "Q6vHf7FNj1yDvX2vggN6kKLgiDP7t7HL5DNpNvK0akyjzfxTFt"
access_token = "164266780-hcTZEl441WUst50TpiRIZtAd3IHmvPDX5tQUDjqx"
access_token_secret = "AIoEwy9AL52AuHRAKesBph3arofQ4AdLwqYRl8tFnehxd"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [7]:
#Create Kafka Producer

ip = 'localhost'
producer = KafkaProducer(bootstrap_servers=[ip+':9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [8]:
#Retrieve TopN Hashtags on the given location

location = 36758 #san francisco
n = 3 #top n parameter
top_n = [x.get("name") for x in api.trends_place(location)[0].get('trends')[0:n]]
print(top_n)

['#WALvFRA', '#JPNvRSA', '#JAPvRSA']


In [4]:
#Define Listener

class ParallelListener(tweepy.StreamListener):
    def __init__(self,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        super(ParallelListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['date'] = str(status.created_at)
            data['tweet'] = status.text
            json_data = json.dumps(data)
            producer.send("test",json_data)
            print(json_data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch all listener together

listener = ParallelListener(time_limit=1000)
myStream = tweepy.Stream(auth = api.auth, listener=listener)
myStream.filter(track=top_n, is_async=False, languages=["en"])

KeyboardInterrupt: 

In [11]:
#Define Listener

class PipelineListener(tweepy.StreamListener):
    def __init__(self,tag,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.tag=tag
        super(PipelineListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['tag'] = self.tag
            data['date'] = str(status.created_at).split(" ")[0]#Keep the date, remove time
            data['tweet'] = status.text
            producer.send("DIC",data)
            print(data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch listener in series

time_limit = 1000

for _ in range(0,5):
    listeners = []
    streams = []
    for i in range(0,n):
        print(top_n[i])
        listeners.append(PipelineListener(top_n[i],time_limit=time_limit))
        streams.append(tweepy.Stream(auth = api.auth, listener=listeners[i]))
        streams[i].filter(track=[top_n[i]], is_async=False, languages=["en"])
        streams[i].disconnect()

#WALvFRA
{'tag': '#WALvFRA', 'date': '2019-10-20', 'tweet': 'RT @giteau_rugby: Brave effort from the French.. a moment of madness effectively cost them a semi final 🤷🏼\u200d♂️🤦🏼\u200d♂️\n\nHuge congrats to @Welsh…'}
{'tag': '#WALvFRA', 'date': '2019-10-20', 'tweet': 'RT @Seasaver: Rugby World Cup #EndJapanWhaling🇯🇵 campaign https://t.co/oU4w1UWI6p #WALvFRA #JPNvRSA #RWC2019'}
{'tag': '#WALvFRA', 'date': '2019-10-20', 'tweet': 'RT @steve_l15: Didn’t miss him\n#WALvFRA #RWC2019 https://t.co/3GbdPee0pB'}
{'tag': '#WALvFRA', 'date': '2019-10-20', 'tweet': "RT @ITVRugby: It had to be him!\n\nYellow-carded earlier, Moriarty may just have rescued Wales' World Cup here...\n\nTMO checks and is happy\n\n#…"}
{'tag': '#WALvFRA', 'date': '2019-10-20', 'tweet': 'RT @michaelsheen: That was the wrong match to be watching holding a baby that hasn’t slept all night but goes off at half-time. It was like…'}
{'tag': '#WALvFRA', 'date': '2019-10-20', 'tweet': 'RT @rugbyworldcup: Highlights: @WelshRugbyUni

KeyboardInterrupt: 